In [31]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import fiona
import rasterio as rio
from rasterio.plot import show
import cartopy.crs as ccrs
from rasterio.mask import mask
import os

%matplotlib inline

In [53]:
#set directories
bare_earth = r"H:\Metro-Van\2021\DEM\Baseline_Master_Dataset_WGS84\revised_jan27\21_3008_00_snowfree_merged_wgs84_utm10_itrf08_epoch2002_dz20cm_1mDEM.tif"
STM_directory = r"H:\Metro-Van\2021\DEM\Co-registered_Snow_DTM\revised_jan28"
lidar_extent = r"H:\Metro-Van\2021\GIS\lidar_boundary\lidar_bounds.shp"
area = "VAN"
phase = "P01"
area_phase = area + "_" + phase
out_name = area_phase + "_sd.tif"
out_directory = r"H:\Metro-Van\2021\GIS\snow_depth_rasters\1m_jan28_coreg"

In [49]:
os.chdir(STM_directory)
STM = area_phase + "_coregSTM.tif"

In [20]:
#open bare earth
with rio.open(bare_earth) as src:
    bare_earth = src.read(1)
    bare_earth_meta = src.meta
    print(bare_earth_meta)
    

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 14396, 'height': 22504, 'count': 1, 'crs': CRS.from_epsg(32610), 'transform': Affine(1.0, 0.0, 492554.0,
       0.0, -1.0, 5495031.0)}


In [ ]:
def snow_depth_raster(bare_earth, STM, area_phase, STM):
    

In [21]:
os.chdir(STM_directory)
with rio.open(STM) as src:
    STM = src.read(1)
    STM_meta = src.meta
    print(STM_meta)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 11811, 'height': 22504, 'count': 1, 'crs': CRS.from_epsg(32610), 'transform': Affine(1.0, 0.0, 492554.0,
       0.0, -1.0, 5495031.0)}


In [28]:
#if the rasters are different sizes then need to clip:
#extract the cooridnates of the sub basin shapefile 
def coords (lidar_extent):
    with fiona.open(lidar_extent) as shapefile:
        for feature in shapefile:
            shapes = [feature["geometry"]]
    return shapes

mask_coords = coords(lidar_extent)
display(mask_coords)

[{'type': 'Polygon',
  'coordinates': [[(501861.0, 5473762.0),
    (501859.490234375, 5473762.490234375),
    (501848.1611328125, 5473769.732421875),
    (501817.509765625, 5473790.509765625),
    (501720.490234375, 5473855.490234375),
    (501717.509765625, 5473857.509765625),
    (501629.490234375, 5473916.490234375),
    (501626.509765625, 5473918.509765625),
    (501535.490234375, 5473979.490234375),
    (501532.509765625, 5473981.509765625),
    (501441.490234375, 5474042.490234375),
    (501438.509765625, 5474044.509765625),
    (501353.490234375, 5474101.490234375),
    (501350.509765625, 5474103.509765625),
    (501316.7734375, 5474126.328125),
    (501337.509765625, 5474189.490234375),
    (501338.490234375, 5474192.509765625),
    (501379.509765625, 5474316.490234375),
    (501380.490234375, 5474319.509765625),
    (501421.509765625, 5474443.490234375),
    (501422.490234375, 5474446.509765625),
    (501462.509765625, 5474567.490234375),
    (501463.490234375, 5474570.5097656

In [50]:
def open_masked_raster_array(tif):
    with rio.open(tif) as src:
        raster_image, raster_transform = rio.mask.mask(src, mask_coords, crop=True)
        raster_image = raster_image.squeeze()
        raster_meta = src.meta
        return [raster_image, raster_meta]

In [54]:
be = (open_masked_raster_array(bare_earth))[0]
meta = (open_masked_raster_array(bare_earth))[1]
STM = (open_masked_raster_array(STM))[0]
#STM_all = open_masked_raster_array(STM)


In [65]:
be[be == -9999] = np.nan
STM[STM == -3.4028235e+38] = np.nan
snow_depth = STM - be
display(be)
display(STM)
display(snow_depth)
snow_depth[snow_depth == -3.4028235e+38] = np.nan


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

array([[-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       ...,
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38],
       [-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
        -3.4028235e+38, -3.4028235e+38, -3.4028235e+38]], dtype=float32)

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [66]:
out_directory = r"H:\Metro-Van\2021\GIS\snow_depth_rasters\1m_jan28_coreg"
out_name = area_phase + "_sd.tif"



os.chdir(out_directory)
with rio.open(out_name, "w", **meta) as dst:
    dst.write(snow_depth, 1)